# MODEL PREDICTION

We will predict the results of prediction on the test images provided in the competition using the best models generated from each of the following approaches 

Approaches followed for model creation
1. Creating the Self designned CNN from scratch 
2. Using the Vgg16 architecture as a base and fine tuning the last layer

In [1]:
import os
import json
from keras.models import load_model
import pandas as pd
import pickle
import numpy as np
import shutil

from keras.preprocessing import image                  
from tqdm.notebook import tqdm
from PIL import ImageFile                            


In [2]:
BASE_MODEL_PATH = os.path.join(os.getcwd(),"model")
TEST_DIR = os.path.join(os.getcwd(),"csv_files","test.csv")
PREDICT_DIR = os.path.join(os.getcwd(),"pred_dir")
PICKLE_DIR = os.path.join(os.getcwd(),"pickle_files")
JSON_DIR = os.path.join(os.getcwd(),"json_files")

In [3]:
if not os.path.exists(PREDICT_DIR):
    os.makedirs(PREDICT_DIR)
else:
    shutil.rmtree(PREDICT_DIR)
    os.makedirs(PREDICT_DIR)
if not os.path.exists(JSON_DIR):
    os.makedirs(JSON_DIR)

# Prediction using-->Self Trained Model (CNN Scratch)

In [4]:
BEST_MODEL = os.path.join(BASE_MODEL_PATH,"self_trained","distracted-21-1.00.hdf5")
model = load_model(BEST_MODEL)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 128)       32896     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 256)       131328    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 256)        0

In [5]:
data_test = pd.read_csv(os.path.join(TEST_DIR))
#testing on the only 10000 images as loading the all test images requires ram>8gb
data_test = data_test[:1000] 
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   FileName   1000 non-null   object
 1   ClassName  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [6]:
with open(os.path.join(PICKLE_DIR,"labels_list.pkl"),"rb") as handle:
    labels_id = pickle.load(handle)
print(labels_id)
# labels_id = {'c1': 0, 'c6': 1, 'c5': 2, 'c8': 3, 'c3': 4, 'c0': 5, 'c7': 6, 'c2': 7, 'c4': 8, 'c9': 9}

{'c8': 0, 'c5': 1, 'c6': 2, 'c4': 3, 'c0': 4, 'c2': 5, 'c1': 6, 'c7': 7, 'c3': 8, 'c9': 9}


In [7]:
from PIL import ImageFile
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import load_img

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(64, 64))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

ImageFile.LOAD_TRUNCATED_IMAGES = True  
test_tensors = paths_to_tensor(data_test.iloc[:,0]).astype('float32')/255 - 0.5

  0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
ypred_test = model.predict(test_tensors,verbose=1)
ypred_class = np.argmax(ypred_test,axis=1)

id_labels = dict()
for class_name,idx in labels_id.items():
    id_labels[idx] = class_name
print(id_labels)

for i in range(data_test.shape[0]):
    data_test.iloc[i,1] = id_labels[ypred_class[i]]

32/32 [==============================] - 3s 77ms/step
{0: 'c8', 1: 'c5', 2: 'c6', 3: 'c4', 4: 'c0', 5: 'c2', 6: 'c1', 7: 'c7', 8: 'c3', 9: 'c9'}


In [9]:
#to create a human readable and understandable class_name 
class_name = dict()
class_name["c0"] = "SAFE_DRIVING"
class_name["c1"] = "TEXTING_RIGHT"
class_name["c2"] = "TALKING_PHONE_RIGHT"
class_name["c3"] = "TEXTING_LEFT"
class_name["c4"] = "TALKING_PHONE_LEFT"
class_name["c5"] = "OPERATING_RADIO"
class_name["c6"] = "DRINKING"
class_name["c7"] = "REACHING_BEHIND"
class_name["c8"] = "HAIR_AND_MAKEUP"
class_name["c9"] = "TALKING_TO_PASSENGER"


with open(os.path.join(JSON_DIR,'class_name_map.json'),'w') as secret_input:
    json.dump(class_name,secret_input,indent=4,sort_keys=True)


In [10]:
# Load JSON data
with open(os.path.join(JSON_DIR, 'class_name_map.json')) as secret_input:
    info = json.load(secret_input)

# Destination directory
dest_dir = "C:/Users/Deepak L/Music/Distracted-Driver-Detection-master/pred_dir"

# Iterate over each row in the DataFrame
for i in range(data_test.shape[0]):
    # Extract file name without extension
    file_name = os.path.basename(data_test.iloc[i, 0]).split(".")[0]
    # Extract class name from JSON and create new file name
    new_name = f"{file_name}_{info[data_test.iloc[i, 1]]}.jpg"
    # Source file path
    src_file = data_test.iloc[i, 0]
    # Destination file path
    dest_file = os.path.join(dest_dir, new_name)
    
    # Check if file exists before copying
    if os.path.exists(src_file):
        shutil.copy(src_file, dest_file)
    else:
        print(f"File not found: {src_file}")
